In [1]:
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import gymnasium as Env
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

from src.environments.fx_environment import FxTradingEnv

Todo:

Create ABC

Tests:

1. Unknown currency in initial portfolio
2. Missing currency in initial portfolio
3. Portfolio value in base
4. Portfolio weights == 1

In [4]:
current_porfolio = {
    "usd": 100_000,
    "eur": 100_000,
    "jpy": 100_000,
    # "sgd": 100_000,
}

In [5]:
historical_data = pd.read_parquet("../data/FX_data.parquet.gzip")

In [6]:
historical_data["2011-01-01":]

ccy,eurjpy,eurusd,sgdjpy,usdjpy,usdsgd
timestamp,,,,,
2011-01-03 09:00:00,108.810,1.33460,63.440,81.180,1.28470
2011-01-03 09:01:00,108.810,1.33460,63.450,81.180,1.28420
2011-01-03 09:02:00,108.810,1.33430,63.460,81.550,1.28440
2011-01-03 09:03:00,108.820,1.33400,63.480,81.570,1.28420
2011-01-03 09:04:00,108.820,1.33390,63.480,81.580,1.28430
...,...,...,...,...,...
2024-12-31 16:54:00,162.825,1.03546,115.108,157.224,1.36508
2024-12-31 16:55:00,162.831,1.03554,115.057,157.219,1.36507
2024-12-31 16:56:00,162.801,1.03539,115.040,157.211,1.36508


In [8]:
fx_env = FxTradingEnv(
    initial_portfolio=current_porfolio,
    historical_prices=historical_data[["eurjpy", "eurusd", "usdjpy"]],
    start_datetime=pd.Timestamp("2011-01-03 09:00:00"),
)

In [9]:
fx_env.preprocess_data()

C:\Users\Ivan\rl_trading\notebooks\..\src\environments\data_processing.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_data[reverse_rate] = 1 / historical_data[fx]


In [10]:
fx_env.existing_currency_pairs

['eurjpy', 'eurusd', 'usdjpy']

In [11]:
fx_env._convert_portfolio_to_base_ccy()

{'usd': 100000.0, 'eur': 133460.0, 'jpy': 1231.830500123183}

In [13]:
fx_env.action_space

Box(-1.0, 1.0, (3,), float32)

In [12]:
fx_env.observation_space

Box(-inf, inf, (15,), float32)